In [1]:
%%capture
!pip install sentence-transformers==3.0.1

In [114]:
import json
import pandas as pd

from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk import ngrams

from typing import List, Union, Optional
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score

import re
from tqdm import tqdm

In [115]:
class EmbeddingModel:
    def __init__(self) -> None:
        pass

    def vectorize(self, form: dict) -> torch.Tensor:
        pass


class TextVectorizer_v2:
    def __init__(self, model_checkpoint: str) -> None:

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model_checkpoint).to(self.device)

    def __preprocess(self, texts: list[str]) -> list[str]:
        """
        Закрытый метод предобработки текста

        Аргументы:
            texts: list[str]

        Возвращает:
            list[str]
        """
        return [txt.replace("\n", "") for txt in texts]


    def vectorize(self, form: Union[dict, list]) -> torch.Tensor:
        """
        Метод векторизации текста

        Аргументы:
            form: dict

        Возвращает:
            torch.Tensor (N, 1024)
        """

        if isinstance(form, dict):
            sentences = self.__preprocess([form[key] for key in tuple(form.keys())[1:]])

            with torch.no_grad():
                sentence_embeddings = self.model.encode(
                    sentences, 
                    convert_to_tensor=True, 
                    normalize_embeddings=True,
                    show_progress_bar=False
                ).to(self.device).cpu()

            return {key: sentence_embeddings[i] for i, key in enumerate(tuple(form.keys())[1:])}

        elif isinstance(form, list):
            sentences = self.__preprocess(form)
            with torch.no_grad():
                sentence_embeddings = self.model.encode(
                    sentences, 
                    convert_to_tensor=True, 
                    normalize_embeddings=True,
                    show_progress_bar=False
                ).to(self.device).cpu()

            return sentence_embeddings

        return None


def get_distance(vec_1: torch.Tensor, vec_2: torch.Tensor) -> float:
    """
    Метод получения расстояния между двумя векторами

    Аргументы:
        vec_1: torch.Tensor
        vec_2: torch.Tensor

    Возвращает:
        float
    """
    if len(vec_1) != 0 and len(vec_1) != 0:
        scores = (vec_1 @ vec_2.T) * 100
        return scores
    else: 
        return torch.tensor([[]])


In [116]:
model = TextVectorizer_v2("intfloat/multilingual-e5-large-instruct")

In [117]:
form_1 = {
    'name' : 'Денис',
    'about_me': "Люблю AI и ML",
    'about_target': "Ищу NLP DS"
}


form_2 = {
    'name': 'Ярослав',
    'about_me': "Люблю Artificial intelligence и Machine learning",
    'about_target': "Ищу Natural language processing Data Science"
}

form_1_emb = model.vectorize(form_1)
form_2_emb = model.vectorize(form_2)

get_distance(form_1_emb["about_target"], form_2_emb["about_me"])

tensor(88.8692)

# Чтение данных

In [127]:
with open('/kaggle/input/ai-prod-hack/main_resumes.json', 'r', encoding='utf-8') as file:
    main_resumes = json.load(file)

with open('/kaggle/input/ai-prod-hack/razmetka.json', 'r', encoding='utf-8') as file:
    razmetka = json.load(file)

In [128]:
cv_info_list = [
    ' '.join(
        [
            rez['position'],
            rez['age'],
            rez['gender'],
            rez['job_search_status'],
            ' '.join(rez['tags']),
            ' '.join(rez['jobs'])
        ]
    )
    for rez in main_resumes
]
cv_about_list = [rez['about'] for rez in main_resumes]
about_me_list = [rez['about_me'] for rez in main_resumes]
about_target_list = [rez['about_target'] for rez in main_resumes]

In [129]:
full_about_me_vec = torch.mean(
    torch.stack(
        [
            model.vectorize(about_me_list),
            model.vectorize(cv_info_list),
            model.vectorize(cv_about_list),
        ]
    ), 
    dim=0
)

about_target_vec = model.vectorize(about_target_list)

# ROC AUC

In [130]:
roc_auc_list = []
for el in tqdm(razmetka):
    index = el['index']
    about_target = about_target_vec[index]

    target = np.concatenate(
        (np.ones(len(el['pos'])),
        np.zeros(len(el['neg'])))
    )
    pos_dist = get_distance(
        about_target,
        full_about_me_vec[el['pos']]
    )
    neg_dist = get_distance(
        about_target,
        full_about_me_vec[el['neg']]
    )
    proba = torch.cat([pos_dist, neg_dist])
    
    try: 
        roc_auc_list.append(roc_auc_score(target, proba))
    except Exception as e:
        print(e)
        pass

100%|██████████| 200/200 [00:00<00:00, 478.09it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


In [133]:
print(sum(roc_auc_list)/len(roc_auc_list))

0.7490461005825515


In [134]:
min_indices = [
    i for i, v in enumerate(roc_auc_list)
    if v in sorted(roc_auc_list)[0:10]
]
min_indices

[0, 1, 10, 18, 34, 37, 47, 61, 74, 197]

In [135]:
roc_auc_list[0]

0.552584670231729

# Анализ анкеты с низкой метрикой

In [147]:
el = razmetka[0]
about_target = about_target_vec[index]
print(main_resumes[0]['about_target'])
about_target_vec[index]

target = np.concatenate(
    (np.ones(len(el['pos'])),
    np.zeros(len(el['neg'])))
)

pos_dist = get_distance(
    about_target,
    full_about_me_vec[el['pos']]
)

neg_dist = get_distance(
    about_target,
    full_about_me_vec[el['neg']]
)
proba = torch.cat([pos_dist, neg_dist])

Я ищу человека, обладающего опытом в сфере продаж и аналитики, особенно в области прогнозирования продаж и сбора данных. Он/она свободно владеет английским языком на уровне Intermediate, а также имеет опыт работы с различными системами, в том числе  AReS, TRAX, Sales Works, Power BI и SAP Business Objects. Кроме того, он/она имеет инженерное образование и опыт работы с CAD-программами, такими как AutoCAD, Mathcad, Creo Elements/Pro и SolidWorks.  Мне необходим такой человек для участия в проекте по разработке системы прогнозирования продаж для нашей компании. 



In [153]:
rec_df = pd.DataFrame(
    {
        'target': target,
        'proba': proba,
        'about_me': (
            [main_resumes[idx]['about_me'] for idx in el['pos']]
            + [main_resumes[idx]['about_me'] for idx in el['neg']]
        ),
        'about': (
            [main_resumes[idx]['about'] for idx in el['pos']]
            + [main_resumes[idx]['about'] for idx in el['neg']]
        ),
        'cv_info': (
            [
                ' '.join(
                    [
                        main_resumes[idx]['position'],
                        main_resumes[idx]['age'],
                        main_resumes[idx]['gender'],
                        main_resumes[idx]['job_search_status'],
                        ' '.join(main_resumes[idx]['tags']),
                        ' '.join(main_resumes[idx]['jobs'])
                    ]
                )
                for idx in el['pos']
            ]
            + [
                ' '.join(
                    [
                        main_resumes[idx]['position'],
                        main_resumes[idx]['age'],
                        main_resumes[idx]['gender'],
                        main_resumes[idx]['job_search_status'],
                        ' '.join(main_resumes[idx]['tags']),
                        ' '.join(main_resumes[idx]['jobs'])
                    ]
                )
                for idx in el['neg']
            ]
        )

    }
)

In [155]:
with pd.option_context('display.max_rows', 5,
                       'display.max_columns', 10,
                       'display.max_colwidth', None):
    display(rec_df.sort_values(by='proba', ascending=False).head(5))

target      proba  \
39     0.0  85.245651   
46     0.0  85.242424   
14     0.0  85.054878   
4      1.0  84.947281   
24     0.0  84.795631   

                                                                                                                                                                                                                                                                                                                                                                             about_me  \
39              Обожаю копаться в данных, искать закономерности и строить прогнозы - чувствую себя немного детективом. В свободное время увлекаюсь фотографией, особенно люблю ловить интересные ракурсы городской архитектуры. Недавно начал осваивать 3D-моделирование, хочу создавать визуализации для своих проектов. Всегда рад пообщаться с коллегами и узнать что-то новое! \n   
46                                                                            Я тот самый человек, который обожает находить неочевидные связи в данных. В свободное от моделей и нейросетей время увлекаюсь астрофотографией – пытаюсь поймать в объектив красоту далеких галактик. Пришел сюда, чтобы послушать интересные доклады и, конечно же, пообщаться с единомышленниками. \n   
14                Обожаю покопаться в данных, найти там что-то интересное и визуализировать так, чтобы даже моя бабушка поняла. В свободное время увлекаюсь скалолазанием –  помогает держать мозг в тонусе. А ещё люблю готовить экзотические блюда, экспериментируя с машинным обучением для подбора идеальных сочетаний вкусов.  Открыт к новым знакомствам и интересным проектам!   
4            В свободное от работы время люблю погонять на велосипеде по лесу – отличный способ разгрузить голову после сложного алгоритма. А еще увлекся 3D-печатью, печатаю всякие полезные мелочи для дома и велосипеда. Интересуюсь новыми технологиями в области машинного обучения, особенно в сфере робототехники. Всегда рад пообщаться с коллегами и узнать что-то новое! \n   
24  В свободное время увлекаюсь астрофотографией, пытаюсь поймать в объектив самые далёкие галактики. Недавно начал осваивать игру на гитаре - пока получается не очень, но соседи пока терпят. Обожаю путешествовать на машине, особенно по местам, где не ловит мобильная связь. Ищу единомышленников для совместных проектов и интересных дискуссий на тему науки и технологий. \n   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  about  \
39  Имею успешный опыт в ра

In [156]:
with pd.option_context('display.max_rows', 5,
                       'display.max_columns', 10,
                       'display.max_colwidth', None):
    display(rec_df.loc[rec_df['target']==1].head(5))

,target,proba,about_me,about,cv_info
0,1.0,80.861015,"В свободное время я увлекаюсь астрономией - космос всегда манил своей неизведанностью, поэтому по вечерам частенько можно застать меня с телескопом на крыше. А еще я учусь играть на укулеле - маленькой, но такой звонкой гавайской гитаре. Это помогает расслабиться после рабочего дня и создает уютную атмосферу. Всегда рада новым знакомствам и интересным беседам! \n","Level of English Intermediate\r Driving license category B. \r I worked as an assistant to the organizer of the VIII St. Petersburg Internet Conference 2013.\r I study English, I try to improve my level.\r I like to travel","Analyst 32 Female Actively searching for a job Английский язык MS PowerPoint Driving Licence B MS Office 1С: Документооборот AutoCAD Mathcad Creo Elements/Pro SolidWorks Power BI SAP Business Objects SOPs specialist/Региональный эксперт по поддержке процессов отдела продаж: Creating sales value regional Key clients and distributors forecaste by SKU.\r Supported current promotion plan in TLC format\r Сollecting information about sales plans from distributors (SI & SO) and analytics of current trends\r Support for Ad-hoc requests from demand planning and SOPs (collection and processing of comments on the performance of the region / new launches / performance of brands / channels and other requests)\r \r Sales department systems supporting (AReS, TRAX, Sales Works) Design engineer category 2: Managed the project to develop interactive guides\r Design and development of submarine parts.\r Cooperation with the various services of the enterprise\r Coordination of design documents with customers or their representatives"
1,1.0,81.964615,"Работаю с данными, строю модели - без этого сейчас никуда. А в свободное время люблю снимать ночное небо на фотоаппарат: находить удачные ракурсы для съёмки и ловить в кадр созвездия. Всегда рад пообщаться с единомышленниками, да и просто интересными людьми.",,"Data Analyst/Data Scientist Мужчина Активно ищет работу Python Machine Learning Numpy pandas Matplotlib Работа с базами данных Базы данных PostgreSQL Linux Docker Data Science Английский язык Data Scientist: Занимался обработкой панельных данных, задачами causal estimation Вел проект Mars Pet food, рассчитывал элластичность с применением методов машинного обучения для товаров этой категории. Data Analyst в отделе внутренного аудита: 1. Провел и написал собственные валидационные тесты модели постконтроля физических лиц причастных к схемам уклонения от уплаты налогов и схемам отмывания нелегально полученных денежных средств. 2. Провел анализ и предложил оптимальный метод вычисления цепочки мошеннических схем, используя Pandas, и представил результат в виде кластеров, полученных из графов при помощи библиотеки NetworkX Специалист отдела информационных технологий: 1. Обработывал клиентские данные в департаменте интегрированных рисков для дальнейшей передачи в работу отдела моделирования. 2. Создавал витрину данных. Стэк использовавшихся технологий - Python, PostgreSQL, Oracle."
2,1.0,84.014023,"В свободное время люблю погонять на велосипеде по парковым тропинкам, а вечерами с удовольствием просматриваю документалки про космос или историю. Всегда в поиске интересных задач, где можно применить свои аналитические способности, поэтому не упускаю возможности познакомиться с коллегами по цеху и обменяться опытом. Кто знает, может быть, наше случайное знакомство перерастет в совместный проект! \n","Я по первому образованию инженер-программист, а по второму экономист. Много лет работала финансовым контролером, но год назад узнала, что появилась специальность на стыке моих навыков (программирования и любимых циферок) - Data Scientist. Это то, что мне очень нравится. Личные качества: коммуникабельна, ответственна, аналитические способности. Профиль на github: Профиль на kaggle:","Data scientist Женщина Python SQL Numpy Git ML Random Forest Scikit-learn Pandas Gradient Boosting JIRA Big Data Data Analysis Bootstrap MS Excel

# map@k

In [212]:
def apk(recs, rels):
    x = 0
    sum = 0
    for i, rec_id in enumerate(recs):
        is_rel = (rec_id in rels)
        mult = 1 if is_rel else 0
        if is_rel:
            x += 1
        sum += mult * (x / (i+1))

    return sum / min(len(rels), len(recs))



k = 5
all_rels = [rel['pos'] for rel in razmetka]
all_recs = []


for el in tqdm(razmetka):
    index = el['index']
    about_target = about_target_vec[index]

    target_idx_list = list(range(0,200))
    target_idx_list.pop(index)

    pos_dist = get_distance(
        about_target,
        full_about_me_vec[el['pos']]
    )
    neg_dist = get_distance(
        about_target,
        full_about_me_vec[el['neg']]
    )
    proba = torch.cat([pos_dist, neg_dist])

    el_recs = np.argpartition(proba, -k)[-k:]
    all_recs.append(el_recs.tolist())

apk_list = [apk(rec, rel) for rec, rel in zip(all_recs, all_rels)]

print('\n', sum(apk_list)/len(apk_list))

100%|██████████| 200/200 [00:00<00:00, 2023.95it/s]


 0.590075


# precison@k и recall@k

In [211]:
k = 5

def precision(rels, recs, k):
    act_set = set(rels)
    pred_set = set(recs[:k])
    result = len(act_set & pred_set) / float(k)
    return result

def recall(rels, recs, k):
    act_set = set(rels)
    pred_set = set(recs[:k])
    result = len(act_set & pred_set) / float(len(act_set))
    return result

precision_list = [precision(rel, rec, k) for rel, rec in zip(all_rels, all_recs)]
recall_list = [recall(rel, rec, k) for rel, rec in zip(all_rels, all_recs)]

print(f'Mean precision@{k} = {sum(precision_list)/len(precision_list)}')
print(f'Mean recall@{k} = {sum(recall_list)/len(recall_list)}')

Mean precision@5 = 0.6419999999999996
Mean recall@5 = 0.04051863870189532
